In [1]:
import os
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, model_selection

import time
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.models import resnet18
import numpy as np
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device:", DEVICE.upper())

# manual random seed is used for dataset partitioning
# to ensure reproducible results across runs
RNG = torch.Generator().manual_seed(42)

Running on device: CUDA


In [2]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

In [3]:
# CIFAR 10 dataset
normalize = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

# Train data
train_set = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=normalize
)
# Train loader
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=2)

# Test data
# we split held out data into test and validation set
test_set = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=normalize
)
test_loader = DataLoader(test_set, batch_size=256, shuffle=True, num_workers=2)

100%|██████████| 170498071/170498071 [00:12<00:00, 13188017.09it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
np.where(np.array(train_set.targets) == 1)[0].shape

(5000,)

In [5]:
# Choose random forget indecies from cars
# Index of class
class_index = 1
class_set = np.where(np.array(train_set.targets) == 1)[0]
# Percantage of whole data ( from class )
amount = 0.1 # 10 %
amount_int = class_set.shape[0] * amount

# Get indeces
forget_idx = np.random.choice(class_set, int(amount_int))

# construct indices of retain from those of the forget set
forget_mask = np.zeros(len(train_set.targets), dtype=bool)
forget_mask[forget_idx] = True
retain_idx = np.arange(forget_mask.size)[~forget_mask]

# split train set into a forget and a retain set
forget_set = torch.utils.data.Subset(train_set, forget_idx)
retain_set = torch.utils.data.Subset(train_set, retain_idx)

forget_loader = torch.utils.data.DataLoader(
    forget_set, batch_size=256, shuffle=True, num_workers=2
)
retain_loader = torch.utils.data.DataLoader(
    retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
)


In [6]:
dataloaders = {
    "train": train_loader,
    "val": test_loader
}

dataset_sizes = {"train": len(train_set), "val": len(test_set)}
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(DEVICE)
                    labels = labels.to(DEVICE)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [17]:
model_ft = resnet18(weights=None, num_classes=10)
# num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 10
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
# model_ft.fc = nn.Linear(num_ftrs, 10)

model_ft = model_ft.to(DEVICE)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [18]:

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=15)

Epoch 0/14
----------
train Loss: 1.3797 Acc: 0.5043
val Loss: 1.1638 Acc: 0.5866

Epoch 1/14
----------
train Loss: 0.9742 Acc: 0.6572
val Loss: 0.9782 Acc: 0.6557

Epoch 2/14
----------
train Loss: 0.7955 Acc: 0.7217
val Loss: 0.9502 Acc: 0.6741

Epoch 3/14
----------
train Loss: 0.6601 Acc: 0.7668
val Loss: 0.8870 Acc: 0.6997

Epoch 4/14
----------
train Loss: 0.5535 Acc: 0.8049
val Loss: 0.9101 Acc: 0.7031

Epoch 5/14
----------
train Loss: 0.4575 Acc: 0.8372
val Loss: 0.8126 Acc: 0.7381

Epoch 6/14
----------
train Loss: 0.3671 Acc: 0.8693
val Loss: 0.8807 Acc: 0.7344

Epoch 7/14
----------
train Loss: 0.1551 Acc: 0.9525
val Loss: 0.7870 Acc: 0.7758

Epoch 8/14
----------
train Loss: 0.0804 Acc: 0.9780
val Loss: 0.8682 Acc: 0.7738

Epoch 9/14
----------
train Loss: 0.0464 Acc: 0.9896
val Loss: 0.9669 Acc: 0.7718

Epoch 10/14
----------
train Loss: 0.0252 Acc: 0.9953
val Loss: 1.0843 Acc: 0.7668

Epoch 11/14
----------
train Loss: 0.0133 Acc: 0.9980
val Loss: 1.2156 Acc: 0.7684

Ep

In [31]:

def unlearning(
    net,
    retain_loader,
    forget_loader,
    val_loader):
    """Simple unlearning by finetuning."""



    class CustomCrossEntropyLoss(nn.Module):
        def __init__(self, class_weights=None):
            super(CustomCrossEntropyLoss, self).__init__()
            self.class_weights = class_weights

        def forward(self, input, target):
            # Compute the standard cross-entropy loss
            ce_loss = nn.functional.cross_entropy(input, target)

            # Apply class weights to the loss if provided
            if self.class_weights is not None:
                # Calculate the weights for each element in the batch based on the target
                weights = torch.tensor([self.class_weights[i] for i in target], device=input.device)
                ce_loss = torch.mean(ce_loss * weights)

            return ce_loss



    # Define the vision_confuser function
    def vision_confuser(model, std = 0.6):
        for name, module in model.named_children():
            if hasattr(module, 'weight'):
                if 'conv' in name:

                    actual_value = module.weight.clone().detach()
                    new_values = torch.normal(mean=actual_value, std=std)
                    module.weight.data.copy_(new_values)

    vision_confuser(net)

    epochs = 4

    w = 0.05

    class_weights = [1, w, w, w, w, w, w, w, w, w]
    criterion = CustomCrossEntropyLoss(class_weights)


    optimizer = optim.SGD(net.parameters(), lr=0.0007,
                      momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=epochs)

    net.train()
    i=0

    for ep in range(epochs):
        i=0
        net.train()
        for sample in retain_loader:
            inputs = sample[0]
            targets = sample[1]
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

        if (ep == epochs-2):
            vision_confuser(net , 0.005) # increase model robustness before last training epoch

        scheduler.step()

    net.eval()


In [32]:
%%time
model_ft_forget = unlearning(model_ft, retain_loader, forget_loader, test_loader)

CPU times: user 48.7 s, sys: 1.48 s, total: 50.2 s
Wall time: 1min 10s


In [12]:
type(forget_loader)

torch.utils.data.dataloader.DataLoader